In [1]:
import numpy as np
X_adv = np.load('Adv_samples_1.npy')

In [2]:
X_adv.shape

(22544, 123)

In [3]:
X_test_scaled = np.load('X_test_scaled.npy')
Y_test = np.load('y_test.npy')

In [4]:
print(X_test_scaled.shape, Y_test.shape)

(22544, 123) (22544, 5)


In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_adv, X_test_scaled, test_size=0.2, random_state=4)

c:\programdata\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
from keras.layers import Input, Dense 
from keras.models import Model, load_model

Using TensorFlow backend.


In [7]:
encoding_dim = 64
input_dim = X_adv.shape[-1]

In [8]:
input_dim

123

In [9]:
def auto_encoder_model(input_dim, encoding_dim) :
    
    input_layer = Input(shape = (input_dim,))
    
    encoder = Dense(encoding_dim, activation = 'tanh')(input_layer)
    encoder = Dense(int(encoding_dim/2), activation="relu")(encoder)
    
    decoder = Dense(int(encoding_dim/2), activation='tanh')(encoder)
    decoder = Dense(input_dim, activation='relu')(decoder)
    
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    autoencoder.summary()
    return autoencoder


In [10]:
denoiser = auto_encoder_model(input_dim, encoding_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 123)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                7936      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 123)               4059      
Total params: 15,131
Trainable params: 15,131
Non-trainable params: 0
_________________________________________________________________


In [11]:
denoiser.fit(X_train, y_train, epochs=20, batch_size=128, validation_data = (X_test, y_test), )

Train on 18035 samples, validate on 4509 samples
Epoch 1/20
18035/18035 [==============================] - 4s 232us/step - loss: 0.0244 - acc: 0.0963 - val_loss: 0.0089 - val_acc: 0.1519
Epoch 2/20
18035/18035 [==============================] - 1s 71us/step - loss: 0.0067 - acc: 0.1458 - val_loss: 0.0055 - val_acc: 0.1007
Epoch 3/20
18035/18035 [==============================] - 1s 74us/step - loss: 0.0050 - acc: 0.1012 - val_loss: 0.0046 - val_acc: 0.1060
Epoch 4/20
18035/18035 [==============================] - 1s 79us/step - loss: 0.0042 - acc: 0.1335 - val_loss: 0.0040 - val_acc: 0.1138
Epoch 5/20
18035/18035 [==============================] - 1s 76us/step - loss: 0.0038 - acc: 0.1346 - val_loss: 0.0038 - val_acc: 0.1016
Epoch 6/20
18035/18035 [==============================] - 2s 90us/step - loss: 0.0036 - acc: 0.1493 - val_loss: 0.0036 - val_acc: 0.1000
Epoch 7/20
18035/18035 [==============================] - 2s 88us/step - loss: 0.0035 - acc: 0.1497 - val_loss: 0.0035 - val_acc

In [12]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_test_pred = loaded_model.predict_classes(X_test_scaled)

In [15]:
from keras.utils import np_utils
X_test_pred = np_utils.to_categorical(X_test_pred)

In [16]:
print(X_test_pred)

[[ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]


In [17]:
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(Y_test, X_test_pred)
print(accuracy_test)

0.837961320085


In [18]:
X_adv_pred = loaded_model.predict_classes(X_adv)

In [19]:
X_adv_pred = np_utils.to_categorical(X_adv_pred)

In [20]:
accuracy_adv = accuracy_score(Y_test, X_adv_pred)
print(accuracy_adv)

0.540942157559


In [21]:
X_adv_denoised = denoiser.predict(X_adv)

In [22]:
X_adv_denoised.shape

(22544, 123)

In [23]:
X_adv_denoised

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.98536086],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.99815893],
       [ 0.        ,  0.        ,  0.        , ...,  1.0247333 ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.0076071 ,
         0.        ,  0.71454358],
       [ 0.        ,  0.        ,  0.        , ...,  0.99055123,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [24]:
X_adv_denoised_pred = loaded_model.predict_classes(X_adv_denoised)

In [25]:
X_adv_denoised_pred = np_utils.to_categorical(X_adv_denoised_pred, num_classes = 5)
print(X_adv_denoised_pred.shape)

(22544, 5)


In [26]:
accuracy_denoised_adv = accuracy_score(Y_test, X_adv_denoised_pred)
print(accuracy_denoised_adv)

0.830952803407
